# Trading News and Corporate Actions with NLP

<a href="" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


<a href="" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Algo-trading on corporate actions by leveraging NLP. A replicationa and enhancement of the paper: *Trade the Event: Corporate Events Detection for News-Based Event-Driven Trading (Zhou et al., Findings 2021)*.

We will perform the following steps:
1. Domain adaptation for financial articles by finetuning a BERT model with Masked Language Model (MLM) training on financial news and encyclopedia data. *Zhou et al.* utilized human annotators to label news articles with an event. 
1. Bi-Level Event Detection: At Token-Level we detect events using a sequence labeling approach. At the higher Article-Level we will augment the corpus with 'CLS' token's embedding or the entire article's, and concatenate the lower level tokens.
1. Recognize security Ticker, using string matching algorithm to recognize tickers within articles.
1. Create trading signals on the identified tickers.

```bibtex
@inproceedings{zhou-etal-2021-trade,
    title = "Trade the Event: Corporate Events Detection for News-Based Event-Driven Trading",
    author = "Zhou, Zhihan  and
      Ma, Liqian  and
      Liu, Han",
    editor = "Zong, Chengqing  and
      Xia, Fei  and
      Li, Wenjie  and
      Navigli, Roberto",
    booktitle = "Findings of the Association for Computational Linguistics: ACL-IJCNLP 2021",
    month = aug,
    year = "2021",
    address = "Online",
    publisher = "Association for Computational Linguistics",
    url = "https://aclanthology.org/2021.findings-acl.186",
    doi = "10.18653/v1/2021.findings-acl.186",
    pages = "2114--2124",
}
```

In [ ]:
"""
%pip install matplotlib
%pip install tensorflow
%pip install shutil
%pip install yfinance
%pip install pyarrow
%pip install tqdm
%pip install pip install transformers
"""

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if 'KAGGLE_KERNEL_RUN_TYPE' in os.environ:
    print('Running in Kaggle...')
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

    DATA_DIR = "/kaggle/input/DATASET"
    IN_KAGGLE = True
else:
    IN_KAGGLE = False
    DATA_DIR = "./data/"

In [ ]:
import numpy as np
import math
import shutil
import yfinance as yf
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from datetime import datetime
import tensorflow as tf

from tqdm import tqdm

if ('COLAB_TPU_ADDR' in os.environ and IN_COLAB) or (IN_KAGGLE and 'TPU_ACCELERATOR_TYPE' in os.environ):
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
elif len(tf.config.list_physical_devices('GPU')) > 0:
    strategy = tf.distribute.MirroredStrategy()
    gpus = tf.config.experimental.list_physical_devices('GPU')
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
    finally:
        print("Running on", len(tf.config.list_physical_devices('GPU')), "GPU(s)")
else:
    strategy = tf.distribute.get_strategy()
    print("Running on CPU")

print("Number of accelerators:", strategy.num_replicas_in_sync)
os.getcwd()

# Wrangling the Data

Our corpus will be processed and labelled to 11 types of corporate events: 
1. Acquisition(A)
1. Clinical Trial(CT)
1. Regular Dividend(RD)
1. Dividend Cut(DC)
1. Dividend Increase(DI)
1. Guidance Increase(GI)
1. New Contract(NC)
1. Reverse Stock Split(RSS)
1. Special Dividend(SD)
1. Stock Repurchase(SR)
1. Stock Split(SS).
1. No event (O)

Articles are structured as follows:

```json
'title': 'Title',
'text': 'Text Body',
'pub_time': 'Published datetime',
'labels': {
    'ticker': 'Security symbol',
    'start_time': 'First trade after article published',
    'start_price_open': 'The "Open" price at start_time',
    'start_price_close': 'The "Close" price at start_time',
    'end_price_nday': 'The "Close" price at the last minute of the following 1-3 trading day. If early than 4pm ET its the same day. Otherwise, it refers to the next trading day.', 
    'end_time_1-3day': 'The time corresponds to end_price_1day',
    'highest_price_nday': 'The highest price in the following 1-3 trading', 
    'highest_time_nday': 'The time corresponds to highest_price_1-3day',
    'lowest_price_nday': 'The lowest price in the following 1-3 trading day',
    'lowest_time_nday': 'The time corresponds to lowest_price_1-3day',
}
```

## BERT Classifier

Built on top of a pretrained BERT (Bidirectional Encoder Representations from Transformers).BERT is an industry tested transformer-based model, pre-trained on a large corpus of text to generate contextual embeddings for input sequences.

We will use a pre-trained Google BERT-Base Cased: with 12-layers + 768-hidden, 12-heads , and 110M parameters. This is the base model used in *Zhou et al. (2021)*.

The architecture can be summarized in 3 componets:
1. Input embeddings + attention masks for the preTrained BERT model. Bert applies transformer blocks with self-attention (attention captures language structures). The model outputs embedding sequences (last layer from BERT NxH) and a pooled summary derived from the first 'CLS' token(a 1XH vector).
1. The sequence outputs (NxH vector) is passed through dense layers and dropouts for the first NER classification, this maps the high-DIM outputs to logits. Padding of unknown tokens helps the model focus on the tasks.
1. NER logits are flattened and concatenated with the pooled summaries to form a new feature vector (NxH + H). The vector is passed again through dense and dropout layers to classify the event as one of the 11 identified (O is ignored).

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense
import tensorflow_text as text
from transformers import TFBertModel, BertConfig, BertTokenizerFast

MODEL_DIR = 'google-bert/bert-base-cased'
MAX_LEN = 256

In [ ]:


class BertForBilevelClassification(TFBertModel):
    def __init__(self, config, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_labels = config.num_labels
        self.max_seq_length = config.max_seq_length
        self.bert = TFBertModel(config)

        self.final_dropout1 = Dropout(config.hidden_dropout_prob)
        self.final_classifier1 = Dense(2048, input_shape=(config.hidden_size + self.max_seq_length * self.num_labels,))
        self.final_dropout2 = Dropout(config.hidden_dropout_prob)
        self.final_classifier2 = Dense(self.num_labels - 1)

        self.seq_dropout = Dropout(config.hidden_dropout_prob)
        self.ner_dropout = Dropout(config.hidden_dropout_prob)

        self.ner_classifier1 = Dense(2048, input_shape=(config.hidden_size,))
        self.ner_classifier2 = Dense(self.num_labels)
        self.dropout1 = Dropout(config.hidden_dropout_prob)
        self.dropout2 = Dropout(config.hidden_dropout_prob)

    def call(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, seq_labels=None, ner_labels=None, output_attentions=None, output_hidden_states=None, return_dict=None, training=False):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask,
                            inputs_embeds=inputs_embeds,
                            output_attentions=output_attentions,
                            output_hidden_states=output_hidden_states,
                            return_dict=return_dict,
                            training=training)

        sequence_output = self.ner_dropout(outputs[0], training=training)
        ner_logits = self.ner_classifier1(sequence_output)
        ner_logits = self.dropout1(ner_logits, training=training)
        ner_logits = self.ner_classifier2(ner_logits)

        pad_pred = tf.zeros([self.num_labels], dtype=ner_logits.dtype)
        pad_pred[-1] = 1
        ner_logits = tf.where(tf.broadcast_to(attention_mask[:, :, None], ner_logits.shape) == 0, pad_pred, ner_logits)

        final_input = tf.reshape(ner_logits, [ner_logits.shape[0], -1])
        final_input = tf.concat([outputs[1], final_input], axis=1)

        seq_logits = self.final_dropout1(final_input, training=training)
        seq_logits = self.final_classifier1(seq_logits)
        seq_logits = self.final_dropout2(seq_logits, training=training)
        seq_logits = self.final_classifier2(seq_logits)

        outputs = (ner_logits,) + (seq_logits,) + outputs[2:]

        if ner_labels is not None:
            loss_fct = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
            ner_loss = loss_fct(ner_labels, ner_logits)

            seq_loss_fct = tf.keras.losses.BinaryCrossentropy(from_logits=True)
            seq_loss = seq_loss_fct(seq_labels[:, :-1], seq_logits)
            total_loss = ner_loss + seq_loss
            outputs = (total_loss,) + outputs

        return outputs

tokenizer = BertTokenizerFast.from_pretrained(MODEL_DIR)
config = BertConfig.from_pretrained(MODEL_DIR)
config.num_labels = 12
config.max_seq_length = MAX_LEN
model = BertForBilevelClassification.from_pretrained(MODEL_DIR, config=config)
model.eval()

# Conclusion

TODO

## References

- [Zhou, Zhihan, Liqian Ma, and Han Liu. "Trade the event: Corporate events detection for news-based event-driven trading." arXiv preprint arXiv:2105.12825 (2021).](https://aclanthology.org/2021.findings-acl.186)
- [Hugging Face Transformers APIs](https://github.com/huggingface/transformers)
- [Hugging Face Model Repository and Spaces](https://huggingface.co/models)
- [Devlin, Jacob, et al. "Bert: Pre-training of deep bidirectional transformers for language understanding." arXiv preprint arXiv:1810.04805 (2018).](https://arxiv.org/abs/1810.04805)
- [Google Pre-trained BERT Models.](https://github.com/google-research/bert?tab=readme-ov-file)

## Github

Article and code available on [Github](https://github.com/adamd1985/news-based-event-driven_algotrading)

Kaggle notebook available [here]()

Google Collab available [here]()

## Media

All media used (in the form of code or images) are either solely owned by me, acquired through licensing, or part of the Public Domain and granted use through Creative Commons License.

## CC Licensing and Use

<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.